In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. Yo

In [2]:
import utils

In [3]:
import altair as alt

In [4]:
pd.set_option("display.max_columns", 100)


In [6]:
itp_id = 300
analysis_date = '2022-05-07'

In [7]:
gtfs_daily = (
        tbl.views.gtfs_schedule_fact_daily_trips()
        >> filter(_.calitp_itp_id == itp_id)
        >> filter(_.service_date == analysis_date)
        >> filter(_.is_in_service == True)
        >> collect()
    )

In [8]:
rt = query_sql(
        f"""
        SELECT * FROM `cal-itp-data-infra-staging.natalie_views.gtfs_rt_distinct_trips`
        WHERE date = '{analysis_date}'
        """
    )

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [9]:
gtfs_daily.head()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
0,4100255323842349957,-5585888450776936120,880900,3480,300,0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,28,28,67200,69420,0.616667
1,4100255323842349957,3883255093367516135,882173,3487,300,0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,30,30,62700,64596,0.526667
2,4100255323842349957,-3566541802341487484,882202,3487,300,0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,32,32,45791,47640,0.513611
3,4100255323842349957,-8929781102916408381,880960,3480,300,0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,33,33,39960,42180,0.616667
4,4100255323842349957,-2569246778384901640,882691,3490,300,0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,34,34,59940,62100,0.600000


In [10]:
gtfs_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870 entries, 0 to 869
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   feed_key                 870 non-null    int64  
 1   trip_key                 870 non-null    int64  
 2   trip_id                  870 non-null    object 
 3   route_id                 870 non-null    object 
 4   calitp_itp_id            870 non-null    int64  
 5   calitp_url_number        870 non-null    int64  
 6   service_id               870 non-null    object 
 7   service_date             870 non-null    object 
 8   service_indicator        870 non-null    object 
 9   service_start_date       870 non-null    object 
 10  service_end_date         870 non-null    object 
 11  service_inclusion        0 non-null      object 
 12  service_exclusion        0 non-null      object 
 13  is_in_service            870 non-null    bool   
 14  calitp_extracted_at      8

In [11]:
rt.head()

,calitp_itp_id,calitp_url_number,date,trip_id,vehicle_id
0,300,0,2022-05-07,882162,1301
1,300,0,2022-05-07,882096,1301
2,300,0,2022-05-07,882158,1301
3,300,0,2022-05-07,882114,1301
4,300,0,2022-05-07,882099,1301


In [12]:
#right: rt 
#left: scheudle 
join = (pd.merge(gtfs_daily, rt, how='outer', on='trip_id', indicator='have_rt'))


In [13]:
join.have_rt.value_counts()

both          608
right_only    395
left_only     266
Name: have_rt, dtype: int64

In [14]:
join["first_departure"] = pd.to_datetime(join.trip_first_departure_ts, unit='s').dt.time


In [15]:
join

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date,vehicle_id,have_rt,first_departure
0,4.100255e+18,-5.585888e+18,880900,3480,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,28.0,28.0,67200.0,69420.0,0.616667,NaN,NaN,NaN,NaN,left_only,18:40:00
1,4.100255e+18,3.883255e+18,882173,3487,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,30.0,30.0,62700.0,64596.0,0.526667,NaN,NaN,NaN,NaN,left_only,17:25:00
2,4.100255e+18,-3.566542e+18,882202,3487,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,32.0,32.0,45791.0,47640.0,0.513611,300.0,0.0,2022-05-07,1503,both,12:43:11
3,4.100255e+18,-8.929781e+18,880960,3480,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,33.0,33.0,39960.0,42180.0,0.616667,300.0,0.0,2022-05-07,1335,both,11:06:00
4,4.100255e+18,-2.569247e+18,882691,3490,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,34.0,34.0,59940.0,62100.0,0.600000,300.0,0.0,2022-05-07,1823,both,16:39:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,NaN,NaN,881509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,5317,right_only,NaT
1265,NaN,NaN,881448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,5317,right_only,NaT
1266,NaN,NaN,881516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,5317,right_only,NaT
1267,NaN,NaN,881442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,5317,right_only,NaT


In [16]:
#rt leftover
rt_lo =(join>>filter(_.have_rt=='right_only'))

In [17]:
rt_lo

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date,vehicle_id,have_rt,first_departure
874,NaN,NaN,882162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,1301,right_only,NaT
875,NaN,NaN,882096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,1301,right_only,NaT
876,NaN,NaN,882158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,1301,right_only,NaT
877,NaN,NaN,882114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,1301,right_only,NaT
878,NaN,NaN,882099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,1301,right_only,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,NaN,NaN,881509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,5317,right_only,NaT
1265,NaN,NaN,881448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,5317,right_only,NaT
1266,NaN,NaN,881516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,5317,right_only,NaT
1267,NaN,NaN,881442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,5317,right_only,NaT


In [18]:
#schedule
sched_lo = (join>>filter(_.have_rt=='left_only'))

In [19]:
sched_lo

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date,vehicle_id,have_rt,first_departure
0,4.100255e+18,-5.585888e+18,880900,3480,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,28.0,28.0,67200.0,69420.0,0.616667,NaN,NaN,NaN,NaN,left_only,18:40:00
1,4.100255e+18,3.883255e+18,882173,3487,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,30.0,30.0,62700.0,64596.0,0.526667,NaN,NaN,NaN,NaN,left_only,17:25:00
8,4.100255e+18,1.516650e+18,881122,3481,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,36.0,36.0,80700.0,83220.0,0.700000,NaN,NaN,NaN,NaN,left_only,22:25:00
10,4.100255e+18,-2.414144e+18,880520,3479,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,42.0,42.0,69960.0,72780.0,0.783333,NaN,NaN,NaN,NaN,left_only,19:26:00
11,4.100255e+18,-5.922786e+18,881975,3486,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,44.0,44.0,78300.0,81000.0,0.750000,NaN,NaN,NaN,NaN,left_only,21:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,4.100255e+18,7.598714e+18,882194,3487,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,32.0,32.0,69611.0,71400.0,0.496944,NaN,NaN,NaN,NaN,left_only,19:20:11
866,4.100255e+18,6.060578e+18,881132,3481,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,36.0,36.0,69600.0,72780.0,0.883333,NaN,NaN,NaN,NaN,left_only,19:20:00
867,4.100255e+18,6.932988e+18,880586,3479,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,45.0,45.0,68580.0,71400.0,0.783333,NaN,NaN,NaN,NaN,left_only,19:03:00
868,4.100255e+18,3.630855e+18,880533,3479,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,45.0,45.0,81840.0,83880.0,0.566667,NaN,NaN,NaN,NaN,left_only,22:44:00


In [20]:
rt_stime = query_sql(
        f"""
        SELECT * FROM `cal-itp-data-infra-staging.natalie_views.test_rt_trips_05072022`
        WHERE date = '{analysis_date}'
        """
    )

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [21]:
rt_stime["start_time"] = pd.to_datetime(rt_stime.start_time, unit='s').dt.time


In [22]:
rt_stime

,calitp_itp_id,calitp_url_number,date,trip_id,start_time
0,300,0,2022-05-07,883191,15:40:48
1,300,0,2022-05-07,882394,17:14:40
2,300,0,2022-05-07,881160,18:57:04
3,300,0,2022-05-07,880812,04:05:20
4,300,0,2022-05-07,882407,22:51:44
...,...,...,...,...,...
994,300,0,2022-05-07,882333,14:15:27
995,300,0,2022-05-07,881174,15:32:15
996,300,0,2022-05-07,882316,02:27:11
997,300,0,2022-05-07,881648,20:52:15


In [23]:
#joining the leftover realtime and join

test_sched = (pd.merge(sched_lo, rt_stime, on='trip_id', how='outer', indicator='_merge'))

In [24]:
test_sched

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date_x,vehicle_id,have_rt,first_departure,calitp_itp_id,calitp_url_number,date_y,start_time,_merge
0,4.100255e+18,-5.585888e+18,880900,3480,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,28.0,28.0,67200.0,69420.0,0.616667,NaN,NaN,NaN,NaN,left_only,18:40:00,NaN,NaN,NaN,NaN,left_only
1,4.100255e+18,3.883255e+18,882173,3487,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,30.0,30.0,62700.0,64596.0,0.526667,NaN,NaN,NaN,NaN,left_only,17:25:00,NaN,NaN,NaN,NaN,left_only
2,4.100255e+18,1.516650e+18,881122,3481,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,36.0,36.0,80700.0,83220.0,0.700000,NaN,NaN,NaN,NaN,left_only,22:25:00,NaN,NaN,NaN,NaN,left_only
3,4.100255e+18,-2.414144e+18,880520,3479,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,42.0,42.0,69960.0,72780.0,0.783333,NaN,NaN,NaN,NaN,left_only,19:26:00,NaN,NaN,NaN,NaN,left_only
4,4.100255e+18,-5.922786e+18,881975,3486,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,44.0,44.0,78300.0,81000.0,0.750000,NaN,NaN,NaN,NaN,left_only,21:45:00,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,NaN,NaN,882333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,14:15:27,right_only
1261,NaN,NaN,881174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,15:32:15,right_only
1262,NaN,NaN,882316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,02:27:11,right_only
1263,NaN,NaN,881648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,20:52:15,right_only


In [25]:
test_sched._merge.value_counts()

right_only    999
left_only     266
both            0
Name: _merge, dtype: int64

In [26]:
#leftover schedule
test_rt = (pd.merge(rt_lo, rt_stime, on='trip_id', how='outer', indicator='_merge'))

In [27]:
test_rt

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date_x,vehicle_id,have_rt,first_departure,calitp_itp_id,calitp_url_number,date_y,start_time,_merge
0,NaN,NaN,882162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,1301,right_only,NaT,300,0,2022-05-07,00:41:44,both
1,NaN,NaN,882096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,1301,right_only,NaT,300,0,2022-05-07,04:11:57,both
2,NaN,NaN,882158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,1301,right_only,NaT,300,0,2022-05-07,04:47:44,both
3,NaN,NaN,882114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,1301,right_only,NaT,300,0,2022-05-07,02:17:06,both
4,NaN,NaN,882099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,1301,right_only,NaT,300,0,2022-05-07,23:58:11,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,NaN,NaN,881604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300,0,2022-05-07,13:32:46,right_only
995,NaN,NaN,882333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300,0,2022-05-07,14:15:27,right_only
996,NaN,NaN,881174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300,0,2022-05-07,15:32:15,right_only
997,NaN,NaN,881648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300,0,2022-05-07,20:52:15,right_only


In [28]:
test_rt._merge.value_counts()

right_only    604
both          395
left_only       0
Name: _merge, dtype: int64

In [29]:
test_rt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999 entries, 0 to 998
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   feed_key                 0 non-null      float64 
 1   trip_key                 0 non-null      float64 
 2   trip_id                  999 non-null    object  
 3   route_id                 0 non-null      object  
 4   calitp_itp_id_x          0 non-null      float64 
 5   calitp_url_number_x      0 non-null      float64 
 6   service_id               0 non-null      object  
 7   service_date             0 non-null      object  
 8   service_indicator        0 non-null      object  
 9   service_start_date       0 non-null      object  
 10  service_end_date         0 non-null      object  
 11  service_inclusion        0 non-null      object  
 12  service_exclusion        0 non-null      object  
 13  is_in_service            0 non-null      object  
 14  calitp_ext

In [30]:
(test_rt>>filter(_._merge=='both')
 >>select(_.trip_id,
          _.calitp_itp_id_y,
          _.calitp_url_number_y,
          _.have_rt,
          _.calitp_itp_id,
          _.calitp_url_number,
          _.date_y,
          _.start_time,
          _._merge)
)

,trip_id,calitp_itp_id_y,calitp_url_number_y,have_rt,calitp_itp_id,calitp_url_number,date_y,start_time,_merge
0,882162,300.0,0.0,right_only,300,0,2022-05-07,00:41:44,both
1,882096,300.0,0.0,right_only,300,0,2022-05-07,04:11:57,both
2,882158,300.0,0.0,right_only,300,0,2022-05-07,04:47:44,both
3,882114,300.0,0.0,right_only,300,0,2022-05-07,02:17:06,both
4,882099,300.0,0.0,right_only,300,0,2022-05-07,23:58:11,both
...,...,...,...,...,...,...,...,...,...
390,881509,300.0,0.0,right_only,300,0,2022-05-07,01:53:44,both
391,881448,300.0,0.0,right_only,300,0,2022-05-07,04:53:15,both
392,881516,300.0,0.0,right_only,300,0,2022-05-07,04:01:02,both
393,881442,300.0,0.0,right_only,300,0,2022-05-07,03:10:10,both


In [31]:
test_sched

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date_x,vehicle_id,have_rt,first_departure,calitp_itp_id,calitp_url_number,date_y,start_time,_merge
0,4.100255e+18,-5.585888e+18,880900,3480,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,28.0,28.0,67200.0,69420.0,0.616667,NaN,NaN,NaN,NaN,left_only,18:40:00,NaN,NaN,NaN,NaN,left_only
1,4.100255e+18,3.883255e+18,882173,3487,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,30.0,30.0,62700.0,64596.0,0.526667,NaN,NaN,NaN,NaN,left_only,17:25:00,NaN,NaN,NaN,NaN,left_only
2,4.100255e+18,1.516650e+18,881122,3481,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,36.0,36.0,80700.0,83220.0,0.700000,NaN,NaN,NaN,NaN,left_only,22:25:00,NaN,NaN,NaN,NaN,left_only
3,4.100255e+18,-2.414144e+18,880520,3479,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,42.0,42.0,69960.0,72780.0,0.783333,NaN,NaN,NaN,NaN,left_only,19:26:00,NaN,NaN,NaN,NaN,left_only
4,4.100255e+18,-5.922786e+18,881975,3486,300.0,0.0,1,2022-05-07,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,44.0,44.0,78300.0,81000.0,0.750000,NaN,NaN,NaN,NaN,left_only,21:45:00,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,NaN,NaN,882333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,14:15:27,right_only
1261,NaN,NaN,881174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,15:32:15,right_only
1262,NaN,NaN,882316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,02:27:11,right_only
1263,NaN,NaN,881648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,20:52:15,right_only


In [32]:
(test_sched>>filter(_._merge=='left_only')
 >>select(_.trip_id,
          _.route_id,
          _.route_id,
          _.first_departure,
          _.calitp_itp_id_x,
          _.calitp_url_number_x,
          _.have_rt,
          _.calitp_itp_id,
          _.calitp_url_number,
          _.date_x,
          _.start_time,
          _._merge)
)

,trip_id,route_id,first_departure,calitp_itp_id_x,calitp_url_number_x,have_rt,calitp_itp_id,calitp_url_number,date_x,start_time,_merge
0,880900,3480,18:40:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only
1,882173,3487,17:25:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only
2,881122,3481,22:25:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only
3,880520,3479,19:26:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only
4,881975,3486,21:45:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...
261,882194,3487,19:20:11,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only
262,881132,3481,19:20:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only
263,880586,3479,19:03:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only
264,880533,3479,22:44:00,300.0,0.0,left_only,NaN,NaN,NaN,NaN,left_only


In [33]:
test_sched.sample()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date_x,vehicle_id,have_rt,first_departure,calitp_itp_id,calitp_url_number,date_y,start_time,_merge
400,NaN,NaN,881937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-07,04:39:56,right_only


In [34]:
test_sched>>count(_.route_id)>>arrange(_.n)

,route_id,n
10,3498,2
8,3493,10
9,3494,11
5,3487,12
7,3490,19
4,3486,22
1,3480,23
6,3489,25
3,3485,42
2,3481,48


In [35]:
alt.Chart((test_sched>>count(_.route_id))).mark_bar().encode(
    x='route_id',
    y='n'
)

alt.Chart(...)

In [36]:
test_sched>>select(_.trip_id,
                  _.first_departure)>>filter(_.first_departure.notnull())

,trip_id,first_departure
0,880900,18:40:00
1,882173,17:25:00
2,881122,22:25:00
3,880520,19:26:00
4,881975,21:45:00
...,...,...
261,882194,19:20:11
262,881132,19:20:00
263,880586,19:03:00
264,880533,22:44:00


In [37]:
# alt.Chart((test_sched)).mark_circle().encode(
#     x='route_id',
#     y='first_departure:T'
# )